# Preparing data

To run the models, we need the following input: 
- X_train_res and y_train_res: resampled train data using SMOTE
- X_test and y_test 
- X_train_12: with only the measurements for stage 1 and 2 
- X_test_12: with only the measurements for stage 1 and 2 

Import libraries. 

In [27]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import confusion_matrix 
from collections import Counter

Standardized_complete_dataset is the dataframe I created with all the extracted features from TS fresh. 
It has 111 rows and 129 columns. 

In [28]:
merged_df = pd.read_csv('/Users/dionnespaltman/Desktop/V6/merged_df_04-06-2024.csv', sep=',')
merged_df.drop('Unnamed: 0.1', axis=1, inplace=True)
merged_df.drop('Unnamed: 0', axis=1, inplace=True)

display(merged_df)

,ID,Stage,Gender,Date,Location,Condition,Faintness,Dizziness,Weakness,Lightheadedness,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,23,1.0,2,2020-08-01,1,2,1.0,2.0,2.0,1.0,...,-0.336826,0.440930,-0.608761,-0.600147,-0.356101,0.239263,0.357168,0.546947,0.644834,0.452026
1,24,1.0,2,2020-01-22,1,2,1.0,1.0,1.0,1.0,...,1.774942,0.440930,2.329367,2.326860,0.188255,-0.249122,-0.025248,0.619122,-0.622877,-0.261674
2,25,1.0,2,2020-05-02,1,2,1.0,1.0,1.0,1.0,...,0.522235,0.440930,0.181549,0.373204,-0.399350,0.048667,0.213801,-0.150745,0.132589,0.169110
3,26,1.0,1,2020-06-02,2,1,2.0,2.0,2.0,1.0,...,-0.443079,0.440930,-0.888021,-0.812001,-0.898647,-0.313749,-0.079950,0.538928,-1.056506,-0.422165
4,27,1.0,1,2020-06-02,2,2,1.0,1.0,1.0,1.0,...,1.662959,1.885134,-2.304025,0.532326,2.472222,1.099701,0.935654,0.651200,3.744591,2.026626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,142,1.0,1,2021-05-31,1,3,1.0,1.0,1.0,1.0,...,-0.908121,0.440930,-1.248127,-1.305586,-0.086453,-0.578589,-0.316920,-0.559738,-0.485024,-0.490340
107,143,1.0,1,2021-01-06,1,3,1.0,1.0,1.0,1.0,...,0.879396,0.440930,0.519120,0.782538,-0.858283,-1.485397,-1.415721,-1.137138,-1.944625,-1.908812
108,144,1.0,1,2021-01-06,1,3,1.0,1.0,1.0,1.0,...,-0.617098,0.440930,-0.891987,-0.930731,-0.748169,-0.803177,-0.537660,-0.359251,-0.925818,-0.818200
109,145,1.0,2,2021-02-06,0,1,1.0,1.0,1.0,1.0,...,-0.535881,0.346928,-0.581134,-0.713336,1.189029,0.042223,0.208832,0.065780,0.032361,0.135367


In [29]:
print(list(merged_df.columns))

['ID', 'Stage', 'Gender', 'Date', 'Location', 'Condition', 'Faintness', 'Dizziness', 'Weakness', 'Lightheadedness', 'Fear', 'Tension', 'Stress', 'Physical_sum', 'Psychological_sum', 'Sum_VVR', 'Sum_12', 'Sum_456', 'Sum_4567', 'Sum_1', 'Sum_2', 'VVR_group', 'AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 'AU01_r__mean', 'AU01_r__root_mean_square', 'AU02_r__sum_values', 'AU02_r__variance', 'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__mean', 'AU02_r__root_mean_square', 'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 'AU04_r__mean', 'AU04_r__root_mean_square', 'AU05_r__sum_values', 'AU05_r__variance', 'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__mean', 'AU05_r__root_mean_square', 'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 'AU06_r__mean', 'AU06_r__root_mean_square', 'AU07_r__sum_values', 'AU07_r__variance', 'AU07_r__standard_deviatio

In [31]:
# columns_to_drop = ['ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition', 'Gender', 'Date', 'Stage', 'Location',] 
columns_to_drop = ['ID', 'Stage', 'Gender', 'Age', 'Date', 'Location', 'Condition', 'Weight', 'Length', 'Faintness',
                    'Dizziness', 'Weakness', 'Lightheadedness', 'Fear', 'Tension', 'Stress', 'Nervousness', 'Physical_sum', 
                    'Psychological_sum', 'Sum_VVR', 'Sum_12', 'Sum_456', 'Sum_4567', 'Sum_1', 'Sum_2', 'VVR_group']

print(len(columns_to_drop))


26


These are the columns I use to predict, so all my features. I need these as a list to establish my featurizer. 
I have 102 features from TS fresh and then I added the two VVR measurements from stage 1 and 2. 

In [32]:
columns_ts_fresh = ['AU01_r__sum_values', 'AU01_r__variance', 'AU01_r__standard_deviation', 'AU01_r__maximum', 
                    'AU01_r__mean', 'AU01_r__root_mean_square', 'AU02_r__sum_values', 'AU02_r__variance', 
                    'AU02_r__standard_deviation', 'AU02_r__maximum', 'AU02_r__mean', 'AU02_r__root_mean_square', 
                    'AU04_r__sum_values', 'AU04_r__variance', 'AU04_r__standard_deviation', 'AU04_r__maximum', 
                    'AU04_r__mean', 'AU04_r__root_mean_square', 'AU05_r__sum_values', 'AU05_r__variance',
                    'AU05_r__standard_deviation', 'AU05_r__maximum', 'AU05_r__mean', 'AU05_r__root_mean_square', 
                    'AU06_r__sum_values', 'AU06_r__variance', 'AU06_r__standard_deviation', 'AU06_r__maximum', 
                    'AU06_r__mean', 'AU06_r__root_mean_square', 'AU07_r__sum_values', 'AU07_r__variance', 
                    'AU07_r__standard_deviation', 'AU07_r__maximum', 'AU07_r__mean', 'AU07_r__root_mean_square', 
                    'AU09_r__sum_values', 'AU09_r__variance', 'AU09_r__standard_deviation', 'AU09_r__maximum',
                    'AU09_r__mean', 'AU09_r__root_mean_square', 'AU10_r__sum_values', 'AU10_r__variance', 
                    'AU10_r__standard_deviation', 'AU10_r__maximum', 'AU10_r__mean', 'AU10_r__root_mean_square',
                    'AU12_r__sum_values', 'AU12_r__variance', 'AU12_r__standard_deviation', 'AU12_r__maximum', 
                    'AU12_r__mean', 'AU12_r__root_mean_square', 'AU14_r__sum_values', 'AU14_r__variance', 
                    'AU14_r__standard_deviation', 'AU14_r__maximum', 'AU14_r__mean', 'AU14_r__root_mean_square', 
                    'AU15_r__sum_values', 'AU15_r__variance', 'AU15_r__standard_deviation', 'AU15_r__maximum', 
                    'AU15_r__mean', 'AU15_r__root_mean_square', 'AU17_r__sum_values', 'AU17_r__variance', 
                    'AU17_r__standard_deviation', 'AU17_r__maximum', 'AU17_r__mean', 'AU17_r__root_mean_square', 
                    'AU20_r__sum_values', 'AU20_r__variance', 'AU20_r__standard_deviation', 'AU20_r__maximum', 
                    'AU20_r__mean', 'AU20_r__root_mean_square', 'AU23_r__sum_values', 'AU23_r__variance', 
                    'AU23_r__standard_deviation', 'AU23_r__maximum', 'AU23_r__mean', 'AU23_r__root_mean_square', 
                    'AU25_r__sum_values', 'AU25_r__variance', 'AU25_r__standard_deviation', 'AU25_r__maximum', 
                    'AU25_r__mean', 'AU25_r__root_mean_square', 'AU26_r__sum_values', 'AU26_r__variance', 
                    'AU26_r__standard_deviation', 'AU26_r__maximum', 'AU26_r__mean', 'AU26_r__root_mean_square', 
                    'AU45_r__sum_values', 'AU45_r__variance', 'AU45_r__standard_deviation', 'AU45_r__maximum', 
                    'AU45_r__mean', 'AU45_r__root_mean_square']

print(len(columns_ts_fresh))

102


# Train and test split

OLD: 
In total, we have 104 participants.

I started with a test size of 20%. Then there are 83 people in the train set and 21 in the test set. 
With a test size of 30#, there are 72 people in the train set and 32 in the test set. 

Naturally, we stratify on VVR_group. 

NEW: 
In total, we have 111 participants.

I started with a test size of 20%. Then there are 88 people in the train set and 23 in the test set. 
With a test size of 30% there are 77 people in the train set and 34 in the test set. 

Naturally, we stratify on VVR_group. 

In [34]:
train, test = train_test_split(merged_df, test_size=0.3, random_state=123, stratify=merged_df['VVR_group'])

print(train.shape)
print(test.shape)

(77, 124)
(34, 124)


OLD: Unfortunately, the test set is very small with only 7 people in the high VVR condition. 

NEW: The test set is very small with only 11 people in the high VVR condition. 

In [35]:
columns_to_drop = [ 'ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition', 'Date', 'Gender'] 

X_test = test.drop(columns_to_drop, axis=1)
y_test = test['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_test))

Original dataset shape Counter({0: 23, 1: 11})


# X_test plus gender 

In [36]:
columns_to_drop = [ 'ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Condition', 'Date'] 

X_test_gender = test.drop(columns_to_drop, axis=1)
display(X_test_gender)

,Stage,Gender,Location,Faintness,Dizziness,Weakness,Lightheadedness,Fear,Tension,Stress,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
80,1.0,2,1,1.0,1.0,1.0,1.0,1.0,3.0,1.0,...,-0.166497,-0.729816,2.825468,1.673580,-2.233545,0.289550,0.393906,-0.511621,1.873165,0.899641
62,1.0,1,0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,-0.363370,-0.080351,-0.509222,-0.565219,-0.579429,-0.022861,0.158174,0.466753,-0.568289,-0.075143
67,1.0,2,0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,0.567536,0.440930,0.271424,0.449194,-0.116745,0.791976,0.739793,0.129936,0.675229,0.802541
43,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.187859,0.346928,-0.008672,0.055083,-0.336010,-0.518784,-0.261437,-0.190843,-0.460154,-0.433136
105,1.0,2,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.077072,0.440930,-0.477280,-0.360369,0.855433,0.216467,0.340371,0.202111,0.358639,0.349728
88,1.0,2,1,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,-2.524026,-3.196200,-0.227209,-1.168774,0.725314,-1.876059,-2.751782,-2.979874,-0.123086,-1.708147
24,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-1.183040,0.440930,-1.648922,-1.689376,-1.010749,-0.856792,-0.593692,-0.487562,-0.978133,-0.884178
5,1.0,1,0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,1.024751,1.876588,2.371954,1.922297,0.497954,2.789133,1.865223,2.022527,0.775269,1.882802
79,1.0,1,1,1.0,1.0,1.0,1.0,1.0,2.0,1.0,...,-0.333410,-1.140004,3.326447,1.972013,-2.248705,-1.876059,-2.751782,-3.390605,-3.525827,-3.577127
100,1.0,1,1,1.0,1.0,1.0,1.0,1.0,2.0,2.0,...,0.033116,0.193108,-0.282383,-0.188178,-0.215466,-0.649872,-0.384751,-0.118668,-0.648225,-0.599438


In [37]:
X_test_gender.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_test_gender.csv')

# Applying SMOTE

In [38]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

We apply SMOTE on the train data. The strategy is to make the minority as big as the majority class. 

In [40]:
columns_to_drop = [ 'ID', 'Sum_12', 'Sum_4567', 'Sum_456', 'VVR_group', 'Date'] 

X_train = train.drop(columns_to_drop, axis=1)
y_train = train['VVR_group']

# Print original class distribution
print('Original dataset shape %s' % Counter(y_train))

# Apply SMOTE to the training data with sampling strategy set to 'auto' (default)
sm = SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=5)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Print resampled class distribution
print('Resampled dataset shape %s' % Counter(y_train_res))

Original dataset shape Counter({0: 51, 1: 26})
Resampled dataset shape Counter({1: 51, 0: 51})


In [41]:
display(X_train_res)
X_train_res.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_train_res_gender.csv')

,Stage,Gender,Location,Condition,Faintness,Dizziness,Weakness,Lightheadedness,Fear,Tension,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,1.0,2,0,2,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,...,-0.588472,0.201653,-1.080948,-1.005558,-1.057202,-1.532650,-1.499128,-1.273469,-2.129886,-2.037173
1,1.0,2,2,2,1.000000,1.000000,1.000000,2.000000,1.000000,4.000000,...,0.289276,0.440930,0.189637,0.225341,1.063062,0.406128,0.477464,0.619122,-0.068091,0.354797
2,1.0,2,1,2,1.000000,1.000000,1.000000,1.000000,3.000000,4.000000,...,0.522235,0.440930,0.181549,0.373204,-0.399350,0.048667,0.213801,-0.150745,0.132589,0.169110
3,1.0,1,0,3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.045264,0.440930,-0.462771,-0.331676,-0.090762,-0.747721,-0.481159,0.330422,-0.600511,-0.671622
4,1.0,2,1,1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,-0.535630,0.440930,-0.856850,-0.858808,0.063929,0.829485,0.764250,0.619122,0.779664,0.855063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1.0,1,0,1,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,...,-0.091406,0.230844,-0.433339,-0.347825,0.856151,1.208075,0.975352,0.504982,0.351762,0.932400
98,1.0,1,0,1,1.000000,1.000000,1.000000,1.000000,1.662522,2.325045,...,1.101031,0.440930,1.002760,1.179665,0.040747,0.137506,0.280514,0.535224,-0.397453,0.086512
99,1.0,2,0,1,1.688289,2.376578,1.688289,2.000000,1.688289,2.623422,...,-0.786450,-0.229599,-1.116187,-1.155161,1.166300,0.814992,0.750020,0.243783,0.607627,0.793926
100,1.0,2,0,3,1.479932,1.479932,1.479932,2.439796,2.479932,2.520068,...,0.914189,0.223561,1.081991,1.105477,1.181302,1.038896,0.862400,0.449104,1.422642,1.139505


# Adding class weights

I will add class weights to my models, because of my inbalanced data set. 
https://medium.com/@ravi.abhinav4/improving-class-imbalance-with-class-weights-in-machine-learning-af072fdd4aa4 

Results: 
{0: 0.6428571428571429, 1: 2.25}

NEW: Class weights: {0: 0.7549019607843137, 1: 1.4807692307692308}

In [42]:
import numpy as np

def calculate_class_weights(y):
    unique_classes, class_counts = np.unique(y, return_counts=True)
    total_samples = len(y)
    class_weights = {}

    for class_label, class_count in zip(unique_classes, class_counts):
        class_weight = total_samples / (2.0 * class_count)
        class_weights[class_label] = class_weight

    return class_weights

# Assuming 'y' contains the class labels (0s and 1s) for the binary classification problem
class_weights = calculate_class_weights(y_train)
print("Class weights:", class_weights)


Class weights: {0: 0.7549019607843137, 1: 1.4807692307692308}


# Save X_train_res, y_train_res, X_test, y_test

In [45]:
print(y_test)

80     0
62     0
67     0
43     0
105    0
88     0
24     0
5      0
79     1
100    0
64     0
75     0
3      0
87     0
78     1
36     0
10     0
103    1
27     0
39     1
48     0
32     1
84     1
46     0
47     1
56     0
54     0
7      1
33     0
76     0
18     1
65     0
90     1
99     1
Name: VVR_group, dtype: int64


In [46]:
X_train_res.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_train_res.csv')
y_train_res.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_train_res.csv')
X_test.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_test.csv')
y_test.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_test.csv')

# X_train_12 and X_test_12 

In this set we want to drop all the action units. 

In [47]:
columns = columns_ts_fresh

print(X_train.shape)
print(X_test.shape)

X_train_12 = X_train.drop(columns=columns)
X_test_12 = X_test.drop(columns=columns)

print(X_train_12.shape)
print(X_test_12.shape)

(77, 118)
(34, 116)
(77, 16)
(34, 14)


In [48]:
X_train_12.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_train_12.csv')
X_test_12.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/X_test_12.csv')
y_train.to_csv('/Users/dionnespaltman/Desktop/V6/data_04-06-2024/y_train_12.csv')